In [0]:
%python
By Default all the tables in databricks are delta table 

In [0]:
%python
delta lake: storage framework help you to build lakehouse
- support ACID
- time travel
- schema evolution
- unifies batch and stream 

In [0]:
%python
delta = parquet + delta_logs
iceberg= parquet + metadata 

In [0]:
%python
Internals of delta 
- parquet (data)
+
_delta_logs
    . crc
    . json 

Optimizing

In [0]:
use catalog dev;
use schema default;

In [0]:
create table demo(id int primary key, name string, age int);

In [0]:
describe extended demo; 

In [0]:
alter table demo add constraint check_age check (age > 18)

In [0]:
insert into demo values (1,'Naval',19),(2,'John',20),(3,'Anil',21),(4,'Chetan',16);

In [0]:
select * from demo;

In [0]:
insert into demo values (1,'Naval',21);
insert into demo values (2,'John',20);
insert into demo values (3,'Anil',19);
insert into demo values(4,'Chetan',17); --- failed
insert into demo values(5,'Mrunal',24); -- skipped

In [0]:
select * from demo;

In [0]:
create table ice ( id int) using iceberg;

In [0]:
desc extended ice;

In [0]:
internal of the delta (storage account, lakehouse, s3 or adls)

parquet (data)
+ 
delta_log (metadata)
1. json files

In [0]:
desc history demo

In [0]:
insert into demo values (1,'Naval',21);

In [0]:
insert into demo values (2,'John',20),(3,'Anil',19);

In [0]:
desc detail demo 

In [0]:
delete from demo

In [0]:
select * from demo

## Time travel

In [0]:
select * from demo version as of 2

In [0]:
select * from demo timestamp as of '2025-12-29T05:53:58.000+00:00'

In [0]:
--create table demov2 as 
select * from demo version as of 2

In [0]:
restore table demo to version as of 2

In [0]:
select * from demo

# Schema Evolution

In [0]:
%python
data=([(1,'a',30),(2,'b',40),(3,'c',50)])
schema="id int, name string, age int"
df=spark.createDataFrame(data,schema)

In [0]:
%python
df.display()

In [0]:
%python
df.write.mode("append").saveAsTable("dev.default.demo")

In [0]:
select * from demo

In [0]:
%python
data=([(4,'d',30,'delhi'),(5,'e',40,'Hyd'),(6,'f',50,'banglore')])
schema="id int, name string, age int, city string"
df=spark.createDataFrame(data,schema)

In [0]:
%python
df.write.mode("append").saveAsTable("dev.default.demo")

In [0]:
%python
df.write.mode("append").option("mergeSchema", "true").saveAsTable("dev.default.demo")

In [0]:
%python
data=([(4,'d','M',30,'delhi'),(5,'e','F',40,'Hyd'),(6,'f','F',50,'banglore')])
schema="id int, name string,gender string, age int, city string"
df=spark.createDataFrame(data,schema)

In [0]:
%python
df.write.mode("append").option("mergeSchema", "true").saveAsTable("dev.default.demo")

In [0]:
select id, name, gender, age, city from demo

In [0]:
%python
data=([(7,'d',30,'Ind'),(8,'e',40,'US'),(9,'f',50,'UK')])
schema="id long, name string, age int, city string"
df=spark.createDataFrame(data,schema)

In [0]:
%python
df.write.mode("append").option("mergeSchema", "true").saveAsTable("dev.default.demo")


In [0]:
%python
from pyspark.sql.functions import *
data=([(7,'d',30,'Ind'),(8,'e',40,'US'),(9,'f',50,'UK')])
schema="id long, name string, age int, city string"
df=spark.createDataFrame(data,schema)

df_new=df.withColumn("id",col("id").cast("int"))

In [0]:

.option("mergeSchema", "true")        # ❌ only adds columns
.option("overwriteSchema", "true")    # ❌ only works with overwrite mode
